In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

plt.rc('font', family='Malgun Gothic')

In [2]:
# 맥북 
data = pd.read_csv('/Users/dongyokim/Library/Mobile Documents/com~apple~CloudDocs/2022:1/마케팅 애널리틱스/중간 과제/직방 competition data 4/train.csv')

In [ ]:
# 윈도우 
data = pd.read_csv('C:/Users/Innovation/Desktop/김동요/jupyter/Data/MA/직방 competition data 4/직방 competition data 4/train.csv')

In [3]:
data.head()

,key,apartment_id,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,...,lowest_building_in_sites,heat_type,heat_fuel,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price
0,0,5584,1,200601,11~20,1999,47.43,6,37.585965,127.000231,...,4.0,individual,gas,91120,65.63,46,1.0,1.0,corridor,215000000
1,1,5584,1,200601,11~20,1999,44.37,8,37.585965,127.000231,...,4.0,individual,gas,91119,61.39,10,2.0,1.0,corridor,200000000
2,2,5059,1,200601,11~20,1992,54.70,8,37.580511,127.014016,...,9.0,individual,gas,8430,72.36,201,2.0,1.0,corridor,168000000
3,3,2816,1,200601,11~20,1993,64.66,11,37.580324,127.011788,...,11.0,individual,gas,5839,87.30,284,2.0,1.0,corridor,165000000
4,4,2816,1,200601,11~20,1993,106.62,7,37.580324,127.011788,...,11.0,individual,gas,5836,127.74,112,4.0,2.0,stairway,280000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1601458 entries, 0 to 1601457
Data columns (total 25 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   key                                 1601458 non-null  int64  
 1   apartment_id                        1601458 non-null  int64  
 2   city                                1601458 non-null  int64  
 3   transaction_year_month              1601458 non-null  int64  
 4   transaction_date                    1601458 non-null  object 
 5   year_of_completion                  1601458 non-null  int64  
 6   exclusive_use_area                  1601458 non-null  float64
 7   floor                               1601458 non-null  int64  
 8   latitude                            1601458 non-null  float64
 9   longitude                           1601458 non-null  float64
 10  address_by_law                      1601458 non-null  int64  
 11  total_parki

## 새로운 열 추가 & 데이터 전처리 

In [3]:
round(data.exclusive_use_area * 0.3025, 4)

0          14.3476
1          13.4219
2          16.5468
3          19.5596
4          32.2526
            ...   
1601453    35.8372
1601454    18.1470
1601455    25.7025
1601456    17.9504
1601457    25.6762
Name: exclusive_use_area, Length: 1601458, dtype: float64

In [4]:
data.exclusive_use_area / 3.3058

0          14.347510
1          13.421865
2          16.546676
3          19.559562
4          32.252405
             ...    
1601453    35.837014
1601454    18.146893
1601455    25.702372
1601456    17.950269
1601457    25.676084
Name: exclusive_use_area, Length: 1601458, dtype: float64

In [5]:
# 새로운 열 추가 

# 해당 평형의 방을 이용하는 세대수가  전체 입주 세대수에서 얼마나 차지하는지 나타냄 
household_ratio = data.total_household_count_of_area_type / data.total_household_count_in_sites 
data['해당 평형 이용 세대수 비율'] = household_ratio

# 해당 방의 평수를 나타냄 
feet_3025 = round(data.exclusive_use_area * 0.3025, 4)
data['방 평수'] = feet_3025

# 평당 가격 
price_3025 = round(data.transaction_real_price / feet_3025, 4)
data['평당 가격'] = price_3025

# 공급면적(전용면적 + 주거공용면적)의 평수 
supply_3025 = round(data.supply_area * 0.3025, 4)
data['공급 면적 평수'] = supply_3025

# 계약 년 & 월 
transaction_year = data.transaction_year_month.apply(lambda x: str(x)[0:4])
    data['transcation_year'] = transaction_year + '년'
transaction_month = data.transaction_year_month.apply(lambda x: str(x)[4:6])
data['transcation_month'] = transaction_month + '월'

transaction_date = transaction_year + "-" + transaction_month + "-01"
transaction_date = transaction_date.astype('datetime64')
data['transaction_date'] = transaction_date

# 계약연도와 아파트 준공년도의 차이(아파트 나이)
apartment_age = data.transaction_year_month.apply(lambda x: int(str(x)[0:4])) - data.year_of_completion + 1
data['거래연도 기준 아파트 나이'] = apartment_age

# 아파트 층 수 높이 순위(최고층 - 해당 방의 층 + 1)
floor_rank = data.tallest_building_in_sites - data.floor + 1
data['층 순위'] = floor_rank

# 층 등급 열 추가
rate = []
for i in range(len(data)):
    if data['floor'][i] <= (0.3333 * data['tallest_building_in_sites'][i]): 
        rate.append('Low')
    elif data['floor'][i] >= ((2/3) * data['tallest_building_in_sites'][i]): 
        rate.append('Royal')
    else:
        rate.append('Medium')
data['층 등급'] = np.array(rate).reshape(-1 ,1)

## 데이터 셋 설정 및 구분 

In [27]:
# EDA & 태블로 시각화용 데이터셋 설정 
tb_data = data.copy()

In [ ]:
# 윈도우용 지하철 정보 데이터 
sub2 = pd.read_excel('C:/Users\/Innovation\/Desktop/김동요/jupyter/Data/MA/Subways2.xlsx')

In [11]:
# 맥용 지하철 정보 데이터 
sub2 = pd.read_excel('/Users/dongyokim/Library/Mobile Documents/com~apple~CloudDocs/2022:1/마케팅 애널리틱스/중간 과제/중간과제 데이터/Subways2.xlsx')

In [28]:
sub2.head()

,station_id,latitude,longitude,subway_line,address_by_law,station,main_line
0,1,37.555729,126.972145,"1,4,KJ,AP",1114012000,서울역,1호선
1,2,37.565624,126.976936,"1,2",1114016700,시청역,1호선
2,3,37.570169,126.983099,1,1111012600,종각역,1호선
3,4,37.571570,126.991895,"1,3,5",1111015600,종로3가역,1호선
4,5,37.570988,127.001921,1,1111016300,종로5가역,1호선


In [29]:
# left 조인  
tb_data = pd.merge(tb_data, sub2, left_on='address_by_law', right_on='address_by_law', how='left')

In [30]:
# 하버시안 공식으로 이용해서 지하철역의 위도, 경도와 아파트의 위도, 경도 사이의 거리 측정

from haversine import haversine

dis = []
for i in range(len(tb_data)):
    apart_where = (tb_data.latitude_x[i], tb_data.longitude_x[i])
    sub_where = (tb_data.latitude_y[i], tb_data.longitude_y[i])
    
    a = haversine(apart_where, sub_where)
    dis.append(a)
    
tb_data['지하철 역까지의 거리'] = np.array(dis).reshape(-1 ,1)

In [31]:
print('데이터 세트의 Shape:', tb_data.shape)
print('\n전체 피처의 type: \n', tb_data.dtypes.value_counts())
isnull_series = tb_data.isnull().sum()
print('\nNull 칼럼과 그 건수:\n ', isnull_series[isnull_series > 0].sort_values(ascending=False))

데이터 세트의 Shape: (3197376, 41)

전체 피처의 type: 
 float64           18
int64             13
object             9
datetime64[ns]     1
dtype: int64

Null 칼럼과 그 건수:
  station_id                        367370
latitude_y                        367370
longitude_y                       367370
subway_line                       367370
station                           367370
main_line                         367370
지하철 역까지의 거리                       367370
total_parking_capacity_in_site    183576
front_door_structure               29916
heat_fuel                          16346
heat_type                           2237
room_count                          1925
bathroom_count                      1925
tallest_building_in_sites              9
lowest_building_in_sites               9
층 순위                                   9
dtype: int64


In [32]:
# null이 너무 많은 컬럼과 불필요한 컬럼 삭제
tb_data = tb_data.drop(['transaction_year_month', 'total_parking_capacity_in_site',  'room_id', 'front_door_structure','heat_type', 'heat_fuel'], axis=1)

# null인 행 모두 삭제 
tb_data.dropna(how='any', inplace=True)

tb_data.reset_index(drop=True, inplace=True)

tb_data.shape

(2828072, 35)

In [33]:
tb_data.head()

,key,apartment_id,city,transaction_date,year_of_completion,exclusive_use_area,floor,latitude_x,longitude_x,address_by_law,...,거래연도 기준 아파트 나이,층 순위,층 등급,station_id,latitude_y,longitude_y,subway_line,station,main_line,지하철 역까지의 거리
0,2,5059,1,2006-01-01,1992,54.70,8,37.580511,127.014016,1111017400,...,15,7.0,Medium,381.0,37.580002,127.015182,6,창신역,6호선,0.117411
1,3,2816,1,2006-01-01,1993,64.66,11,37.580324,127.011788,1111017400,...,14,5.0,Royal,381.0,37.580002,127.015182,6,창신역,6호선,0.301313
2,4,2816,1,2006-01-01,1993,106.62,7,37.580324,127.011788,1111017400,...,14,9.0,Medium,381.0,37.580002,127.015182,6,창신역,6호선,0.301313
3,7,9867,1,2006-01-01,2005,84.76,10,37.559200,127.019503,1114016200,...,2,14.0,Medium,15.0,37.565314,127.016249,"2,6",신당역,2호선,0.737813
4,7,9867,1,2006-01-01,2005,84.76,10,37.559200,127.019503,1114016200,...,2,14.0,Medium,72.0,37.554443,127.011002,"3,6",약수역,3호선,0.917269


In [34]:
# 데이터 csv파일로 추출 
tb_data.to_csv('EDA_dataset.csv')